In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
from scipy import stats
import numpy as np
import pickle
import pprint # Pretty Printer!
import warnings
import statistics

Use the tag csv. Check the locatin data to see if there are any commets about using only a left or right mic.
Next, get the tag amplitude per species detection.
Filter for the 'transcribed' sites.
Get the mean amp per song
Since this is detection only data, we can use the task data, somehow merge it with the tag so each detection has one amplitude attached to it. Then, as a condition, change the value of detectoin given the amp.
I'll have to get that condition. It depends on the ARU unit (SM2 or not), openness, and species.
The script to do this is in AbundanceConditionalOccupancy, called distance truncation.ipynb.

# Prepare the data for truncation

In [2]:
raw_Path = r"C:\Users\ilebe\Documents\!Masters!\RETN Community\0_Data\Raw\Wildtrax Limited Percept"
processed_Path = r"C:\Users\ilebe\Documents\!Masters!\RETN Community\0_Data\Processed"
# Set your working directory
site_detections = pd.read_csv(os.path.join(processed_Path, "Limited perceptibility", 'Multi_spp_occupancy.csv'))
site_detections.head()

,location,recording_date_time,latitude,longitude,task_comments,WTSP,TEWA,AMCR,LEYE,WISN,...,BRCR,GCKI,GRYE,FOSP,DOWO,SOSA,DUFL,TOSO,COLO,PAWR
0,H-CC-1-101,2022-06-06 07:51:00,52.621107,-115.168551,NaN,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,H-CC-1-102,2022-06-03 05:53:00,52.617102,-115.161641,NaN,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,H-CC-1-104,2022-06-07 07:50:00,52.618950,-115.155674,NaN,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,H-CC-1-108N,2022-06-24 07:56:00,54.415680,-116.361990,NaN,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,H-CC-1-115N,2022-06-22 08:18:00,53.430128,-117.467584,NaN,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Make a dataframe with the same format as this, but with the amplitudes instead. Can match with the combination of the column and location, from the tags raw data

In [3]:
tags = pd.read_csv(os.path.join(raw_Path, "BU_Harvest_Patch_Retention_-_Community_&_Limited_perceptibility_tag_report.csv"))
# tags.head(3)
# tags.columns
tags = tags[['location', 'recording_date_time', 'species_code', 'left_freq_filter_tag_peak_level_dbfs', 'right_freq_filter_tag_peak_level_dbfs']]

Filter out the non-songs, and check any recordings where there is a comment to use only one mic

In [8]:
tags.head(3)

,location,recording_date_time,species_code,left_freq_filter_tag_peak_level_dbfs,right_freq_filter_tag_peak_level_dbfs
0,H22-CC-0,2022-06-18 06:51:00,LEYE,-33.68,-35.28
1,H22-CC-0,2022-06-18 06:51:00,MAWA,-23.76,-18.15
2,H22-CC-0,2022-06-18 06:51:00,AMRO,-34.91,-37.17


Identify recordings where amp needs to be taken form only one mic

In [5]:
site_detections[site_detections.task_comments.notna()]

,location,recording_date_time,latitude,longitude,task_comments,WTSP,TEWA,AMCR,LEYE,WISN,...,BRCR,GCKI,GRYE,FOSP,DOWO,SOSA,DUFL,TOSO,COLO,PAWR
36,H-RS-1-58N,2022-06-12 07:33:00,53.229932,-116.931810,use right mic only,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49,H-RS-22-47,2021-06-22 06:00:00,56.313000,-116.709176,use left mic only,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
54,H-RS-6-53,2021-06-16 06:00:00,52.196080,-115.336147,Use right mic only,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
65,H22-CC-131,2022-06-12 06:25:00,54.559660,-119.770600,use left mic only,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
109,H23-RS-177,2023-06-22 07:18:00,51.678160,-115.306235,use right mic only,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
test = tags.copy(deep=True)

In [13]:
site_detections['task_comments'] = site_detections['task_comments'].str.lower()
# Find all locations in 'site_detections' where the task comment is "use right mic only"
left_mics_to_update = site_detections.loc[site_detections['task_comments'] == "use right mic only", 'location']
right_mics_to_update = site_detections.loc[site_detections['task_comments'] == "left right mic only", 'location']
# Use Boolean indexing in 'tags' to find all rows with these locations
left_update_mask = tags['location'].isin(left_mics_to_update)
right_update_mask = tags['location'].isin(right_mics_to_update)

# Update the 'left_freq_filter_tag_peak_level_dbfs' in 'tags' where the mask is True
tags.loc[left_update_mask, 'left_freq_filter_tag_peak_level_dbfs'] = tags.loc[left_update_mask, 'right_freq_filter_tag_peak_level_dbfs']
tags.loc[right_update_mask, 'right_freq_filter_tag_peak_level_dbfs'] = tags.loc[right_update_mask, 'left_freq_filter_tag_peak_level_dbfs']

In [10]:
test[test['location'] == 'H23-RS-177']

,location,recording_date_time,species_code,left_freq_filter_tag_peak_level_dbfs,right_freq_filter_tag_peak_level_dbfs
349,H23-RS-177,2023-06-22 07:18:00,BBMA,-23.48,-23.48
350,H23-RS-177,2023-06-22 07:18:00,WAVI,-38.18,-38.18


In [15]:
tags.dropna(inplace=True)
tags['mean_amp'] = tags[['left_freq_filter_tag_peak_level_dbfs', 'right_freq_filter_tag_peak_level_dbfs']].mean(axis=1)

In [16]:
tags[['location', 'recording_date_time', 'species_code', 'mean_amp']].to_csv(os.path.join(processed_Path, "Limited perceptibility", "species_detection_amplitude.csv"), index=False)

Create a dataframe of the amplitude of each *loudest* detection

In [17]:
# Pivot species_amplitude_df to wide format where each species becomes a column
species_amp_pivot = tags.pivot_table(
    index=["location", "recording_date_time"], 
    columns="species_code", 
    values="mean_amp",
    aggfunc='first'  # Assuming there's only one entry per species per location and date
).reset_index()

species_amp_pivot.fillna(value=0, inplace=True)
species_amp_pivot.head(3)

species_code,location,recording_date_time,ALFL,AMCR,AMRE,AMRO,ATTW,BAWW,BBMA,BCCH,...,WETA,WEWP,WISN,WIWA,WIWR,WTSP,WWCR,YBFL,YEWA,YRWA
0,H-CC-1-101,2022-06-06 07:51:00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,-49.37,0.0,0.0,0.000,-38.145,0.0,0.0,0.00
1,H-CC-1-102,2022-06-03 05:53:00,-44.68,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-48.92,0.0,0.00,0.0,0.0,-30.610,0.000,0.0,0.0,0.00
2,H-CC-1-108N,2022-06-24 07:56:00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.00,0.0,0.0,-44.315,0.000,0.0,0.0,-40.67


# Prepare truncated dataframes

In [18]:
# load predictive dataframe from the playback model

covariates_path = "C:/Users/ilebe/Documents/!Masters!/Analysis/0. Data/Processed/covariates"
predicted_amps = pd.read_csv(r"C:\Users\ilebe\Documents\!Masters!\Playback analysis\0. Data\Processed\real data\predicted_distance_amplitudes_500m_all_species.csv")
SM2_status = pd.read_csv(os.path.join(covariates_path, "SM2 status.csv"))
harv_age = pd.read_csv(r"C:\Users\ilebe\Documents\!Masters!\Analysis\AbundanceConditionalOccupancy\Processed Data\Harv_age.csv")

In [19]:
harv_age.loc[harv_age['Year_since_logging']>11, 'forest' ] = 1
harv_age['forest'].fillna(0, inplace=True)
harv_age_sm2 = pd.merge(harv_age, SM2_status, on='location')
harv_age_sm2.head(2)

,location,Year_since_logging,forest,SM2
0,AM-429-NW,6,0.0,1.0
1,BG-6-94,6,0.0,0.0


In [20]:
species_amp_covs = pd.merge(species_amp_pivot, harv_age_sm2, on='location')

Add whether each recording was taken with an SM2 or not

In [21]:
data_to_truncate_sm2 = pd.merge(species_amp_pivot, SM2_status, on='location')

In [22]:
predicted_amps_150 = predicted_amps[predicted_amps['distance'].between(149, 151)]
predicted_amps_150.head()
len(predicted_amps_150.species_code.unique())

15

Map the reference species to the actual species. Will need to delete the speceis fo rwhich this quesotin doesn't make sense

In [43]:
playback_experiment_path = r"C:\Users\ilebe\Documents\!Masters!\Playback analysis"
centroid_species = pd.read_csv(os.path.join(playback_experiment_path, "2. Output/centroid_species.csv"))
cca_groups = pd.read_csv(os.path.join(playback_experiment_path, "2. Output/species_per_cluster.csv"))

In [44]:
cca_groups.head()
cca_group_dict = dict(zip(cca_groups['species'], cca_groups['group']))
centroid_group_dict = dict(zip(centroid_species['group'], centroid_species['species']))
centroid_group_dict

{15: 'ALFL',
 11: 'AMCR',
 4: 'BADO',
 9: 'BOCH',
 6: 'BRBL',
 2: 'CCSP',
 1: 'DEJU',
 7: 'LCSP',
 10: 'MAWR',
 14: 'MGWA',
 13: 'NSWO',
 12: 'VESP',
 8: 'WEME',
 3: 'CSWA',
 5: 'AMRO'}

In [42]:
cca_groups['reference'] = cca_groups['group'].map(centroid_group_dict)
cca_groups.head()
cca_groups['reference'].unique()

array(['ALFL', 'AMCR', 'BRBL', 'LCSP', 'AMRO', 'BADO', 'VESP', 'DEJU',
       'YEWA', 'MAWR', 'BOCH', 'MGWA', 'CCSP', 'WEME', 'NSWO'],
      dtype=object)

In [21]:
cca_groups[['species', 'reference']].to_csv(os.path.join(processed_Path,"Limited perceptibility", "species and references.csv"), index=False)

## Load the reference species as a dict

In [23]:
spp_df = pd.read_csv(os.path.join(processed_Path,"Limited perceptibility", "species and references.csv"))
spp_dict = dict(zip(spp_df['species'], spp_df['reference']))
spp_dict

{'ALFL': 'ALFL',
 'AMCR': 'AMCR',
 'AMGO': 'BRBL',
 'AMRE': 'LCSP',
 'AMRO': 'AMRO',
 'BADO': 'BADO',
 'BAOR': 'VESP',
 'BAWW': 'DEJU',
 'BBWA': 'CSWA',
 'BCCH': 'CSWA',
 'BHCO': 'ALFL',
 'BHGR': 'VESP',
 'BLJA': 'MAWR',
 'BOCH': 'BOCH',
 'BOWA': 'LCSP',
 'BRBL': 'BRBL',
 'BRCR': 'MOWA',
 'BRSP': 'VESP',
 'BTNW': 'CCSP',
 'CAVI': 'AMCR',
 'CCLO': 'MOWA',
 'CCSP': 'CCSP',
 'CEDW': 'BRBL',
 'CHSP': 'AMRO',
 'CMWA': 'CSWA',
 'COLO': 'WEME',
 'CONW': 'DEJU',
 'COYE': 'DEJU',
 'DEJU': 'DEJU',
 'EAPH': 'LCSP',
 'FOSP': 'VESP',
 'GCKI': 'BOCH',
 'GHOW': 'NSWO',
 'GWTE': 'AMRO',
 'HETH': 'DEJU',
 'KILL': 'DEJU',
 'LCSP': 'LCSP',
 'LEFL': 'BOCH',
 'LISP': 'WEME',
 'MAWA': 'BOCH',
 'MAWR': 'MAWR',
 'MCLO': 'VESP',
 'MOBL': 'AMCR',
 'MOCH': 'DEJU',
 'MOWA': 'MOWA',
 'NESP': 'MAWR',
 'NOWA': 'DEJU',
 'NSWO': 'NSWO',
 'OCWA': 'AMRO',
 'OSFL': 'CSWA',
 'OVEN': 'WEME',
 'PAWA': 'DEJU',
 'RBGR': 'VESP',
 'RBNU': 'AMRO',
 'RCKI': 'WEME',
 'REVI': 'MAWR',
 'RWBL': 'DEJU',
 'SAPH': 'ALFL',
 'SAVS': 'AMRO

## Truncate 

In [24]:
predicted_amps_150.shape

(120, 8)

In [25]:
predicted_amps_150_close= predicted_amps[predicted_amps['distance'].between(149.8, 150.2)]
predicted_amps_150_close.shape

(60, 8)

In [26]:
predicted_amps_150_close.rename({"species_code": "reference"}, axis =1, inplace=True)

C:\Users\ilebe\AppData\Local\Temp\ipykernel_19628\1274968650.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predicted_amps_150_close.rename({"species_code": "reference"}, axis =1, inplace=True)


Expand the precited_amps dataframe so each species in our study has a row, and the values are associated with the reference species

In [27]:
predicted_amps_150_close.head()

,Unnamed: 0,distance,reference,BinForest,SM2,lwr,upr,predicted
139,140,150.1002,ALFL,FO,0,-44.058379,-41.639552,-42.848965
639,640,150.1002,AMCR,FO,0,-41.736021,-39.422756,-40.579388
1139,1140,150.1002,BRBL,FO,0,-43.059483,-40.729200,-41.894342
1639,1640,150.1002,LCSP,FO,0,-60.893557,-57.251782,-59.072669
2139,2140,150.1002,AMRO,FO,0,-39.969024,-37.729788,-38.849406


In [28]:
predicted_amps_150_close_all_spp = pd.merge(spp_df, predicted_amps_150_close, on='reference')
predicted_amps_150_close_all_spp.drop(labels="Unnamed: 0", axis=1, inplace=True)
predicted_amps_150_close_all_spp.head()

,species,reference,distance,BinForest,SM2,lwr,upr,predicted
0,ALFL,ALFL,150.1002,FO,0,-44.058379,-41.639552,-42.848965
1,ALFL,ALFL,150.1002,FO,1,-48.536618,-46.000747,-47.268682
2,ALFL,ALFL,150.1002,OP,0,-41.177021,-38.385483,-39.781252
3,ALFL,ALFL,150.1002,OP,1,-45.646772,-42.755166,-44.200969
4,BHCO,ALFL,150.1002,FO,0,-44.058379,-41.639552,-42.848965


In [74]:
predicted_amps_150_close_all_spp.to_csv(os.path.join(processed_Path, "Limited Perceptibility", "truncation_150_data.csv"), index = True)

In [29]:
predicted_amps_150_close_all_spp.shape

(352, 8)

To truncate, make a new dataframe where each species column 

In [30]:
species_amp_covs_copy = species_amp_covs.copy(deep=True)

In [31]:
species_amp_covs_copy.head(2)

,location,recording_date_time,ALFL,AMCR,AMRE,AMRO,ATTW,BAWW,BBMA,BCCH,...,WIWA,WIWR,WTSP,WWCR,YBFL,YEWA,YRWA,Year_since_logging,forest,SM2
0,H-CC-1-101,2022-06-06 07:51:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000,-38.145,0.0,0.0,0.00,8,0.0,1.0
1,H-CC-1-108N,2022-06-24 07:56:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,-44.315,0.000,0.0,0.0,-40.67,13,1.0,0.0


In [32]:
species_amp_covs_copy.to_csv(os.path.join(processed_Path, "Limited Perceptibility", "community_ready_to_truncate.csv"), index = True)

In [33]:
# Cleaning column names for easier processing
community_df = species_amp_covs_copy
truncation_df = predicted_amps_150_close_all_spp.copy(deep = True)
truncation_df.columns = [col.strip() for col in truncation_df.columns]
community_df.columns = [col.strip() for col in community_df.columns]

# Ensure the forest and SM2 columns in community_df are properly formatted
community_df['forest'] = community_df['forest'].astype(float)
community_df['SM2'] = community_df['SM2'].astype(float)

# Define a function to process each row in community_df
def process_row(row, truncation_df):
    # For each species column in community_df
    for species in truncation_df['species'].unique():
        if species in row:
            # Find the matching row in truncation_df
            match = truncation_df[(truncation_df['species'] == species) & 
                                  (truncation_df['BinForest'] == ('FO' if row['forest'] == 1.0 else 'OP')) & 
                                  (truncation_df['SM2'] == int(row['SM2']))]
            if not match.empty and row[species] != 0 and row[species] < match['predicted'].iloc[0]:
                row[species] = 0
            else:
                row[species] = 1 if row[species] != 0 else 0
    return row

# Apply the function to each row in community_df
community_df = community_df.apply(lambda row: process_row(row, truncation_df), axis=1)

# Display the first few modified rows
community_df.head()


,location,recording_date_time,ALFL,AMCR,AMRE,AMRO,ATTW,BAWW,BBMA,BCCH,...,WIWA,WIWR,WTSP,WWCR,YBFL,YEWA,YRWA,Year_since_logging,forest,SM2
0,H-CC-1-101,2022-06-06 07:51:00,0,0,0,0,0.0,0,0.0,0,...,0,0,0,1,0.0,0,0,8,0.0,1.0
1,H-CC-1-108N,2022-06-24 07:56:00,0,0,0,0,0.0,0,0.0,0,...,0,0,0,0,0.0,0,1,13,1.0,0.0
2,H-CC-1-125,2022-06-10 07:49:00,0,0,0,0,0.0,0,0.0,0,...,0,0,0,0,0.0,0,0,11,0.0,1.0
3,H-CC-1-21N,2022-07-05 06:26:00,0,0,0,0,0.0,0,0.0,0,...,0,0,0,0,0.0,0,1,21,1.0,0.0
4,H-CC-1-26,2022-07-03 05:54:00,0,0,0,0,0.0,0,0.0,0,...,0,0,0,0,0.0,0,0,4,0.0,0.0


In [38]:
# Check with species were not found in the reference dataframe
metadata_columns = ['location', 'recording_date_time', 'Year_since_logging', 'forest', 'SM2']
species_columns = [col for col in community_df.columns if col not in metadata_columns]

# Check for any columns where any of the values are less than 0
columns_with_negatives = [col for col in species_columns if any(community_df[col] < 0)]

columns_with_negatives


[]

In [37]:
community_all_accounted = community_df[community_df.columns.difference(columns_with_negatives)]
community_all_accounted.head()

,ALFL,AMCR,AMRE,AMRO,BAWW,BCCH,BHCO,BHVI,BLJA,BOCH,...,WIWA,WIWR,WTSP,WWCR,YEWA,YRWA,Year_since_logging,forest,location,recording_date_time
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,8,0.0,H-CC-1-101,2022-06-06 07:51:00
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,13,1.0,H-CC-1-108N,2022-06-24 07:56:00
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,11,0.0,H-CC-1-125,2022-06-10 07:49:00
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,21,1.0,H-CC-1-21N,2022-07-05 06:26:00
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,4,0.0,H-CC-1-26,2022-07-03 05:54:00


# Ready to do occupancy now!!! YAY

In [39]:
community_all_accounted.to_csv(os.path.join(processed_Path, "Limited perceptibility","Occupancy_truncated_150.csv"), index=False)